In [1]:
import os
import json
from underthesea import sent_tokenize
from langchain_text_splitters import CharacterTextSplitter

class VietnameseTextProcessor:
    def __init__(self, max_length: int = 5000, overlap: int = 500):
        self.max_length = max_length
        self.overlap = overlap
        
    def read_text_files(self, input_folder: str) -> list:
        """Đọc tất cả các file văn bản trong thư mục."""
        all_texts = []
        for filename in os.listdir(input_folder):
            if filename.endswith('.txt'):
                with open(os.path.join(input_folder, filename), 'r', encoding='utf-8') as f:
                    all_texts.append(f.read())
        return all_texts  # Trả về danh sách các văn bản

    def preprocess_text(self, text: str) -> str:
        """Tiền xử lý văn bản: giữ nguyên định dạng và dấu câu."""
        text = text.replace('\r\n', '\n')
        text = '\n'.join(line.strip() for line in text.split('\n'))
        return text
    
    def split_into_sentences(self, text: str) -> list:
        """Tách văn bản thành các câu sử dụng Underthesea."""
        sentences = sent_tokenize(text)
        valid_sentences = [s.strip() for s in sentences if len(s.split()) > 5]
        return valid_sentences
    
    def create_chunks(self, sentences: list) -> list:
        """Ghép các câu thành chunk sử dụng LangChain."""
        splitter = CharacterTextSplitter(
            chunk_size=self.max_length,
            chunk_overlap=self.overlap,
            separator="\n",
            length_function=len
        )
        text = '\n'.join(sentences)
        chunks = splitter.split_text(text)
        return chunks

    def save_to_file(self, chunks: list, output_file: str):
        """Lưu các chunk vào file .json."""
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(chunks, f, ensure_ascii=False, indent=4)  # Lưu dưới dạng JSON

# Sử dụng lớp VietnameseTextProcessor
if __name__ == "__main__":
    processor = VietnameseTextProcessor()
    input_folder = '../data-raw/Thich Nhat Hanh 2'
    output_file = 'output_chunks.json'

    texts = processor.read_text_files(input_folder)
    all_sentences = []
    for text in texts:
        preprocessed_text = processor.preprocess_text(text)
        sentences = processor.split_into_sentences(preprocessed_text)
        all_sentences.extend(sentences)

    chunks = processor.create_chunks(all_sentences)

    # In ra tổng số chunk
    print(f"Tổng số chunk đã tạo: {len(chunks)}")

    # Lưu các chunk vào file JSON
    processor.save_to_file(chunks, output_file)

    print(f"Các chunk đã được lưu vào file: {output_file}")

/Users/phucnt/.pyenv/versions/3.11.10/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tổng số chunk đã tạo: 566
Các chunk đã được lưu vào file: output_chunks.json
